# Popularity based recommender Systems

# Classification Based Recommeder System

# Nearest Neighbour Colaborative Filtering 

![title](collaborativeFiltering.jpg)

# Item Based Collaborative Filtering

![title](recommendation-grid.png)

![title](co-occurrence-matrix.png)

# Model Based Collaborative Filtering (Matrix Factorization)

# Building a Music Recommender System

Crab module in Python used to recommendation

In [1]:
%matplotlib inline

import pandas as pd
from sklearn.cross_validation import train_test_split
import numpy as np
import time
from sklearn.externals import joblib
import Recommenders as Recommenders
#import Evaluation as Evaluation

/Users/melon/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#WE are using here a subset of milion songs data set
triplets = 'https://static.turi.com/datasets/millionsong/10000.txt' #triplets = user_id,song_id and listen_count
song_data = 'https://static.turi.com/datasets/millionsong/song_data.csv' #Info about songs

songs_df1 = pd.read_csv(triplets , delim_whitespace= True , header = None)
songs_df1.columns = ['user_id' , 'song_id' , 'listen_count']

#Read song meta data
songs_df2 = pd.read_csv(song_data)


#Merging the above two data frames to make input data frame for our recommender system
song_df = pd.merge(songs_df1 , songs_df2.drop_duplicates(['song_id']) , on = 'song_id' , how = 'left')

song_df.head()


,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


# Length of data

In [3]:
len(song_df)

2000000

# Creating a subset of the data set

In [4]:
song_df = song_df.head(10000)

In [5]:
#Merge the 'title' column and 'artist_name' column to produce a new column (song)
#Each song can be distinguish by song-name and artist, hence combined

song_df['song'] = song_df['title'].map(str) + '-' + song_df['artist_name']
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove-Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas-Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger-Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations-Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly-Foo Fighters


In [6]:
del song_df['title']
del song_df['artist_name']
song_df.head()

,user_id,song_id,listen_count,release,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,Thicker Than Water,0,The Cove-Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Flamenco Para Niños,1976,Entre Dos Aguas-Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Graduation,2007,Stronger-Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,In Between Dreams,2005,Constellations-Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,There Is Nothing Left To Lose,1999,Learn To Fly-Foo Fighters


# Showing the most popular songs in the dataframe

In [7]:
song_grouped = song_df.groupby(['song']).agg({'listen_count' : 'count'}).reset_index()
song_grouped.head()

,song,listen_count
0,#40-DAVE MATTHEWS BAND,1
1,& Down-Boys Noize,4
2,'97 Bonnie & Clyde-Eminem,2
3,'Round Midnight-Miles Davis,3
4,'Till I Collapse-Eminem / Nate Dogg,6


In [8]:
grouped_sum = song_grouped['listen_count'].sum()
grouped_sum

10000

In [9]:
song_grouped['percentage'] = song_grouped['listen_count'].div(grouped_sum)*100
song_grouped.head()

,song,listen_count,percentage
0,#40-DAVE MATTHEWS BAND,1,0.01
1,& Down-Boys Noize,4,0.04
2,'97 Bonnie & Clyde-Eminem,2,0.02
3,'Round Midnight-Miles Davis,3,0.03
4,'Till I Collapse-Eminem / Nate Dogg,6,0.06


In [10]:
song_grouped.sort_values(['listen_count' , 'song'] , ascending = False  , inplace = True)
song_grouped.head()


,song,listen_count,percentage
3660,Sehr kosmisch-Harmonia,45,0.45
5105,You're The One-Dwight Yoakam,32,0.32
4678,Undo-Björk,32,0.32
3655,Secrets-OneRepublic,28,0.28
1071,Dog Days Are Over (Radio Edit)-Florence + The ...,28,0.28


# Count number of users in the dataframe(Unique Users)

In [11]:
users = song_df['user_id'].unique()  # Returns numpy ndarray 
print(users[:5])
print(len(users))

['b80344d063b5ccb3212f76538f3d9e43d87dca9e'
 '85c1f87fea955d09b4bec2e36aee110927aedf9a'
 'bd4c6e843f00bd476847fb75c47b4fb430a06856'
 '8937134734f869debcab8f23d77465b4caaa85df'
 '969cc6fb74e076a68e36a04409cb9d3765757508']
365


# Counting the number if unique songs in the dataframe

In [12]:
unique_songs = song_df['song'].unique()
print(unique_songs[:5])
print(len(unique_songs))

['The Cove-Jack Johnson' 'Entre Dos Aguas-Paco De Lucia'
 'Stronger-Kanye West' 'Constellations-Jack Johnson'
 'Learn To Fly-Foo Fighters']
5151


# Creating a song recommender

In [13]:
train_data , test_data = train_test_split(song_df , test_size = 0.02 , random_state = 0)
train_data.head(5)

,user_id,song_id,listen_count,release,year,song
6768,e3937c7c32f5b68422808a854a4a7a824ee448a5,SOYOHMP12A6D4F6244,1,Mary Ann meets the Gravediggers and other shor...,2002,Daniel Cowman-Regina Spektor
779,baf47ed8da24d607e50d8684cde78b923538640f,SOCDXUG12B0B8064EA,1,Parklife,1994,End Of A Century-Blur
4277,8dcb524ff75e0ad0f0a80aaccadf7bbaa3b89a41,SOLVMXY12A8C131E36,1,Idealism,2007,Moonlight-Digitalism
1507,484b69dd013df1ec0cfd504886d4f647cb32b08f,SOCXCTS12AC468A380,1,Various Artists/Rough Trade Counter Culture 09,2009,Basic Space-The xx
3776,884209a41deb55df792f074bccf8af1c1c31768b,SOCXMRE12AB018AE72,3,Crystal Castles ( II ),2010,Baptism-Crystal Castles


In [14]:
test_data.head()

,user_id,song_id,listen_count,release,year,song
9394,97e48f0f188e04dcdb0f8e20a29dacb881d80c9e,SOLCAXX12B0B8079A5,1,Rabbit Habits,2008,Mister Jung Stuffed-Man Man
898,a820d2d4f16bbd53be9e41e0417dfb234bfdfba8,SOHMTUC12A58A799D4,2,Cunning Stunts,1992,Ort-Cows
2398,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,SOEUTXF12A6D4FC6F8,2,Muriendo de Amor,0,Suena tu guitarra-Fernando Soto
5906,806ccae96c8ecb1c198482aff785ccd6bbe17143,SOIBSSJ12A8151E066,1,3 Doors Down,2008,Let Me Be Myself-3 Doors Down
2343,8cbb5066924ec788e3fea9a4aae59586f46f38fa,SOPPROJ12AB0184E18,1,Pursuit Of Happiness,2010,Pursuit Of Happiness (nightmare)-Kid Cudi / MG...


# Popularity Based Recommeder Systems

### Creating instance of popularity based recommender class

In [15]:
pr = Recommenders.popularity_recommender_py()
pr.create(train_data , 'user_id' , 'song') 

### Using the popularity model to make some predictions

In [16]:
#test_user = test_data.loc[ 114563 , 'user_id'] 
#Picking up user_id of any index in the test_data and predicting for test_user
#print(test_user)
#pr.recommend(test_user)

In [17]:
test_user2 = users[10]
pr.recommend(test_user2)

,user_id,song,score,Rank
3622,17aa9f6dbdf753831da8f38c71b66b64373de613,Sehr kosmisch-Harmonia,45,1.0
5044,17aa9f6dbdf753831da8f38c71b66b64373de613,You're The One-Dwight Yoakam,32,2.0
4624,17aa9f6dbdf753831da8f38c71b66b64373de613,Undo-Björk,30,3.0
1063,17aa9f6dbdf753831da8f38c71b66b64373de613,Dog Days Are Over (Radio Edit)-Florence + The ...,28,4.0
3617,17aa9f6dbdf753831da8f38c71b66b64373de613,Secrets-OneRepublic,27,5.0
4658,17aa9f6dbdf753831da8f38c71b66b64373de613,Use Somebody-Kings Of Leon,26,6.0
4327,17aa9f6dbdf753831da8f38c71b66b64373de613,The Scientist-Coldplay,26,7.0
3440,17aa9f6dbdf753831da8f38c71b66b64373de613,Revelry-Kings Of Leon,24,8.0
1374,17aa9f6dbdf753831da8f38c71b66b64373de613,Fireflies-Charttraxx Karaoke,24,9.0
1844,17aa9f6dbdf753831da8f38c71b66b64373de613,Horn Concerto No. 4 in E flat K495: II. Romanc...,23,10.0


# Building a personalized Song Recommender

We now build an item-similarity based collaborative filtering model recommender system to give more personalised recommendations.

### Creating an instance of the item-similarity based recommender system class

In [18]:
is_model = Recommenders.item_similarity_recommender_py()
is_model.create(train_data , 'user_id' , 'song') #training our model for the training data

### Using Personalised model to make some recommendations

In [19]:
#print the songs for the user in tha training data
user_id = users[5]
user_items = is_model.get_user_items(user_id)

print("-------------------------------------------------------------------------------------------------------------")
print("Training data songs for the user with user_id = %s" %user_id)
print("-------------------------------------------------------------------------------------------------------------")

for item in user_items:
    print(item)
    
print("-------------------------------------------------------------------------------------------------------------")
print("Recommendation Process")
print("-------------------------------------------------------------------------------------------------------------") 

is_model.recommend(user_id)

-------------------------------------------------------------------------------------------------------------
Training data songs for the user with user_id = 4bd88bfb25263a75bbdd467e74018f4ae570e5df
-------------------------------------------------------------------------------------------------------------
Without Me-Eminem
Missing You-John Waite
Ya Nada Queda-Kudai
16 Candles-The Crests
My Dad's Gone Crazy-Eminem / Hailie Jade
Speechless-Lady GaGa
Push It-Salt-N-Pepa
Ghosts 'n' Stuff (Original Instrumental Mix)-Deadmau5
The Real Slim Shady-Eminem
Forgive Me-Leona Lewis
Just Lose It-Eminem
Somebody To Love-Justin Bieber
Say My Name-Destiny's Child
-------------------------------------------------------------------------------------------------------------
Recommendation Process
-------------------------------------------------------------------------------------------------------------
Number of unique songs for the user: 13
Number of unique songs in the training set: 5089


TypeError: 'set' object does not support indexing

### We can also apply the same model to get similar songs to any other song in the dataframe

In [20]:
is_model.get_similar_items(['U Smile-Justin Bieber'])

Number of unique songs in the training set: 5089


ValueError: Arrays were different lengths: 9800 vs 21